# Function Download >> See To DO Below : ไม่จำเป็นตัวรู้กดโหลดเลยก็ได้
filename = Directory +"/"+ filename+".jpg" #อันนี้รองรับการสร้าง Folder แล้วยัดไปเรื่อยๆ 
ระวังเรื่องการใช้งาน Directory นะ

In [1]:
#!/usr/bin/python

# Note to Kagglers: This script will not run directly in Kaggle kernels. You
# need to download it and run it on your local machine.

# Downloads images from the Google Landmarks dataset using multiple threads.
# Images that already exist will not be downloaded again, so the script can
# resume a partially completed download. All images will be saved in the JPG
# format with 90% compression quality.

import sys, os, multiprocessing, csv
from PIL import Image
from io import BytesIO
from urllib.request import urlopen
import time
import os

def ParseData(data_file):
    csvfile = open(data_file, 'r')
    csvreader = csv.reader(csvfile)
    key_url_list = [line[:3] for line in csvreader]
    return key_url_list[1:]  # Chop off header

import urllib.request

def download_image(url,filename,Directory):
#*************************************** ระวังผิด directory นะ set ก่อนด้วย ****************************
        filename = Directory +"/"+ str(filename) +".jpg" #อันนี้รองรับการสร้าง Folder แล้วยัดไปเรื่อยๆ
#*************************************** ระวังผิด directory นะ set ก่อนด้วย ****************************
        if os.path.exists(filename):
            print('Image %s already exists. Skipping download.' % filename)
            return
        urllib.request.urlretrieve(url,filename)     


In [2]:

#*************************************** ระวังผิด directory นะ set ก่อนด้วย ****************************
 #Use disc directory
#*************************************** ระวังผิด directory นะ set ก่อนด้วย ****************************
def Download_Manager(start,end,Data) :   
    start_time = time.time()
     # label_to_category_dict[str(temp_dir)][1] จะเรียก ชื่อ Landmark จาก Dict มา
    for j in range(start,end): 
        temp_dir = j 
        directory = label_to_category_dict[str(temp_dir)][1]
        print("Load CLass >>",j,directory)
        for i in range(3) :
#                 if ( i == 0  ) : 
#                     base_directory = "D:/Train"
#                     if not (os.path.exists(base_directory)) :
#                         print("Crate train folder >>",base_directory,directory)
#                         os.makedirs(base_directory)
#                 elif (i == 1 ) : 
#                     base_directory = "D:/Test"
#                     if not (os.path.exists(base_directory)) :
#                         print("Crate test folder >>",base_directory,directory)
#                         os.makedirs(base_directory)
                if (i == 2 ) : 
                    base_directory = "D:/Validate"
                    if not (os.path.exists(base_directory)) :
                        print("Crate validate folder >>",base_directory,directory)
                        os.makedirs(base_directory)
                else : 
                    print("error create Test-Train-Valiadate")
                    continue
                for k in range(len(Data[j][i]))  :
                    try :
                        if os.path.exists(base_directory +'/'+ directory): #ใส่รูปไปเรื่อยๆ จนกว่าจะไปเจอ Directory 
                                #print('Image %s already exists. Skipping download.' % base_directory + directory)
                            download_image(Data[j][i].iloc[k][2],Data[j][i].iloc[k][1],base_directory+'/'+ directory)
                        else : 
                            os.makedirs(base_directory +'/'+ directory) #เมื่อต้องสร้างไฟล์ใหม่ ต้องมีการใส่รูป
                            download_image(Data[j][i].iloc[k][2],Data[j][i].iloc[k][1],base_directory+'/'+ directory)
                        if ( k % 200 == 0) : 
                            print("Pass",str(k))
                            end_time = time.time()
                            print(end_time-start_time)
                            start_time = time.time()

                    except KeyboardInterrupt:
                        return
                    except :
                        print("error")
    return True


# Create downloadable file
fomat data เป็น Csv. 3 colurm จะเป็นผมให้มันเป็น แรก 3 line ในคำสั่ง ParseData(path)

In [3]:
csv_download = ParseData('landmark_recognition-master/data/data.csv')

In [4]:
#Play safe for index_boundary tranform list to dict 
label_to_category = ParseData('landmark_recognition-master/data/label_to_category.csv')
label_to_category_dict = {}

for el_lable in  label_to_category :
    label_to_category_dict[el_lable[2]] = [el_lable[0],el_lable[1]]
print(label_to_category_dict["0"]) 

['192931', 'york_river_state_park']


# Data split train-test

In [5]:
import numpy as np 
import pandas as pd
len(csv_download)

91618

In [6]:
#Split data 
#ตัวของข้อมูลนั้นทุกตัวคือคนละ Class 
#Expect ต้องแยกได้ 7 Folder

lst_raw_order = [] #L คือ List ของภาพที่เป็นคนละ Class
for i in range(len(csv_download)) :  # Loop นี้จะแยกและหาตำแหน่ง สุดท้าย ของ Class แต่ละ Class
    if i == len(csv_download)-1 :
        lst_raw_order.append(int(csv_download[i][0]))
        break
    if (csv_download[i][2] != csv_download[i+1][2]) :
        #print(i)
        lst_raw_order.append(int(csv_download[i][0]))
c = 0
# for j in lst_data : 
#     print(c,j,"\n")
#     c += 1 

In [7]:
# raw_data = np.asarray(csv_download)
# lst_raw_data_split = []
# for i in range(100) : 
#     if ( i == 0 ) : 
#         lst_raw_data_split.append(raw_data[:lst_raw_order[i]])
#     else : 
#         lst_raw_data_split.append(raw_data[lst_raw_order[i-1]:lst_raw_order[i]] )
# print(len(lst_raw_data_split))
# #test Miss data in list
# for j in range(100) : 
#     for el_lst in range(1,len(lst_raw_data_split[i])) : 
#         if (lst_raw_data_split[i][el_lst][2] != lst_raw_data_split[i][el_lst][2] ) : 
#             print("error_detect",lst_raw_data_split[1][el_lst])

In [8]:
#Split Data to create CSV
# train_set = []
# test_set  = []
# var_set   = []
# Data = []
# for el_class in lst_raw_data_split :
#     el_class = pd.DataFrame(el_class)
#     train, validate, test = np.split(el_class.sample(frac=1), [int(.8*len(el_class)), int(.9*len(el_class))])
#     train_set.append(train)
#     test_set.append(test)
#     var_set.append(validate)
#     Data.append(list([train,test,validate]))
# #from pandas import DataFrame

In [9]:
#Create CSV
# for j in range(100) :
#     for i in range(3) :
#         temp_dir = j 
#         directory = label_to_category_dict[str(temp_dir)][1]
#         if ( i == 0  ) : 
#             base_directory = "D:/Train"
#             if not (os.path.exists(base_directory)) :
#                 os.makedirs(base_directory)
#             pd.DataFrame(train_set[j]).to_csv("D:/Train/"+str(j)+".csv")
#         elif (i == 1 ) : 
#             base_directory = "D:/Test"
#             if not (os.path.exists(base_directory)) :
#                 os.makedirs(base_directory)
#             pd.DataFrame(test_set[j]).to_csv( "D:/Test/"+str(j)+".csv")
#         elif (i == 2 ) : 
#             base_directory = "D:/Validate"
#             if not (os.path.exists(base_directory)) :
#                 os.makedirs(base_directory)
#             pd.DataFrame(var_set[j]).to_csv("D:/Validate/"+str(j)+".csv")
        

In [7]:
Data = list()
for j in range(100) :
    for i in range(3) :
        temp_dir = j 
        directory = label_to_category_dict[str(temp_dir)][1]
        if ( i == 0  ) :
            temp_train = pd.read_csv("Train/"+str(j)+".csv")
        elif (i == 1 ) : 
            temp_test = pd.read_csv( "Test/"+str(j)+".csv")
        elif (i == 2 ) : 
            temp_val = pd.read_csv("Validate/"+str(j)+".csv")
    Data.append([temp_train,temp_test,temp_val])

In [11]:
Data

[[      Unnamed: 0     0                                                  1  2
  0           1008  1008  https://upload.wikimedia.org/wikipedia/commons...  0
  1           1178  1178  https://upload.wikimedia.org/wikipedia/commons...  0
  2           1830  1830  https://upload.wikimedia.org/wikipedia/commons...  0
  3           1757  1757  https://upload.wikimedia.org/wikipedia/commons...  0
  4           2140  2140  https://upload.wikimedia.org/wikipedia/commons...  0
  5            103   103  https://upload.wikimedia.org/wikipedia/commons...  0
  6            356   356  https://upload.wikimedia.org/wikipedia/commons...  0
  7           2030  2030  https://upload.wikimedia.org/wikipedia/commons...  0
  8            690   690  https://upload.wikimedia.org/wikipedia/commons...  0
  9            808   808  https://upload.wikimedia.org/wikipedia/commons...  0
  10          2258  2258  https://upload.wikimedia.org/wikipedia/commons...  0
  11           137   137  https://upload.wikimedia.o

# Start Download

แปลงเป็น Function ที่สามารถเรียกค่า ของ Directory และจัดการไฟล์

In [12]:
label_to_category_dict

{'0': ['192931', 'york_river_state_park'],
 '1': ['186080', 'osaka_prefectural_flower_garden'],
 '2': ['19605', 'saiful_muluk_lake'],
 '3': ['171683', 'lok_virsa_museum,_islamabad'],
 '4': ['189811', 'lake_como'],
 '5': ['86869', 'mount_rainier_national_park'],
 '6': ['90021', 'rohtas_fort'],
 '7': ['62798', 'enchanted_floral_gardens_of_kula'],
 '8': ['181631', 'leesylvania_state_park'],
 '9': ['10618', 'botanical_garden_jevremovac'],
 '10': ['165596', 'edinburgh_castle'],
 '11': ['165900', 'mount_arapiles'],
 '12': ['145015', 'texas_state_library_and_archives_commission'],
 '13': ['171772', 'university_of_chicago_library'],
 '14': ['67416', 'manassas_national_battlefield_park'],
 '15': ['177194', 'royal_national_park'],
 '16': ['107750', 'chippokes_plantation_state_park'],
 '17': ['137203', 'port_of_casablanca'],
 '18': ['73300', 'iguazu_falls'],
 '19': ['149980', 'khotyn_fortress'],
 '20': ['176018', 'hayravank_monastery'],
 '21': ['184313', 'kahanu_garden'],
 '22': ['16658', 'glasne

In [13]:
def Top_load(Data = Data) : 
    for i in range(23,36):
        Download_Manager(i,i+1,Data)
        print("Batch >>",i," ==> ",label_to_category_dict[str(i)][1],"Complete")
    return True

In [14]:
def Can_load(Data = Data) : 
    for i in range(70,80):
        Download_Manager(i,i+1,Data)
        print("Batch >>",i," ==> ",label_to_category_dict[str(i)][1],"Complete")
    return True

In [15]:
def Poom_load(Data=Data) : 
    for i in range(80,90):
        Download_Manager(i,i+1,Data)
        print("Batch >>",i," ==> ",label_to_category_dict[str(i)][1],"Complete")
    return True

In [16]:
def Pfirst_load(Data=Data) : 
    for i in range(90,99):
        Download_Manager(i,i+1,Data)
        print("Batch >>",i," ==> ",label_to_category_dict[str(i)][1],"Complete")
    return True

# #TO DO 1:::
Load Data ตามชื่อตัวเอง แต่ละคนจะโหลดไฟล์ ประมาณ 10 GB แล้วอัพลง Google Drive นิสิต แล้วมาแชร์กัน

In [17]:
#Top_load()
#Can_load()
#Poom_load()
Pfirst_load()

Load CLass >> 90 sky_meadows_state_park
error create Test-Train-Valiadate
error create Test-Train-Valiadate
Pass 0
4.0537989139556885
Batch >> 90  ==>  sky_meadows_state_park Complete
Load CLass >> 91 lake_constance
error create Test-Train-Valiadate
error create Test-Train-Valiadate
Pass 0
1.9703330993652344
Batch >> 91  ==>  lake_constance Complete
Load CLass >> 92 ellora_caves
error create Test-Train-Valiadate
error create Test-Train-Valiadate
Pass 0
4.5775535106658936
Batch >> 92  ==>  ellora_caves Complete
Load CLass >> 93 neghuts_monastery
error create Test-Train-Valiadate
error create Test-Train-Valiadate
Pass 0
8.41252589225769
Batch >> 93  ==>  neghuts_monastery Complete
Load CLass >> 94 bakhchisaray_palace
error create Test-Train-Valiadate
error create Test-Train-Valiadate
Pass 0
12.769202470779419
Batch >> 94  ==>  bakhchisaray_palace Complete
Load CLass >> 95 ogrodzieniec_castle
error create Test-Train-Valiadate
error create Test-Train-Valiadate
Pass 0
2.1392173767089844
Bat

True